# <img style="float: left; padding-right: 20px; width: 200px" src="https://raw.githubusercontent.com/raxlab/imt2200-data/main/media/logo.jpg">  IMT 2200 - Introducción a Ciencia de Datos
**Pontificia Universidad Católica de Chile**<br>
**Instituto de Ingeniería Matemática y Computacional**<br>
**Semestre 2025-S2**<br>
**Profesor:** Rodrigo A. Carrasco <br>
---

# <h1><center>Clase 04: Trabajo con Datos Semi Estructurados</center></h1>

Este ejercicio busca que los estudiantes aprendan a usar librerias en Python para la gestión y exploración de datos semi estructurados.

## 1. Datos para los ejemplos

Usaremos dos conjuntos de datos para este Notebook:
<ol>
<li>Datos de Yelp sobre restaurantes de Santiago:</li>
    
Estos datos fueron proporcionados por la profesora Paula Aguirre y están en el archivo `restaurants.json` en la carpeta de `data`.

    
<li>Datos de conciertos de la Filarmónica de NY:</li>

La orquesta filarmónica de Nueva York, ha puesto a disposición del público los datos de las más de 20,000 presentaciones que han hecho desde el 7 de diciembre de 1842, contando con una base de más de 180 años de presentaciones. 
    
La base de datos original puede ser accedida en forma interactiva en https://archives.nyphil.org/performancehistory/#program.op y la base completa está disponible en el sitio de GitHub de la Filarmónica: https://github.com/nyphilarchive/PerformanceHistory
</ol>

## 2. Manipulación de datos en formato json

Uno de los formatos preferidos para datos semi estructurados es el formato json. Python cuenta con una libraría especial para trabajar con este tipo de formato: https://docs.python.org/3/library/json.html y nos permite leer y escribir en este formato entre otras cosas.

A continuación cargaremos las librerías necesarias para este ejercicio.

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np
import pandas as pd
import json

In [ ]:
%cd data

## 3. Introducción a json

### 3.1 Creando un set de datos simple

Armemos una lista de curso de alumnos en la universidad por generación, incporporando datos adicionales, como profesores y notas cuando estén estas disponibles. Comenzaremos con un `list` en Python para almacenar cada generación.

In [ ]:
json_list = [
    { 
        'class': 'Year 1', 
        'student count': 35, 
        'room': 'A2',
        'info': {
            'teachers': { 
                'math': 'Emmy Noether', 
                'physics': 'Richard Feynman' 
            }
        },
        'students': [
            { 
                'name': 'Mary', 
                'sex': 'F', 
                'grades': { 'math': 75, 'physics': 98 } 
            },
            { 
                'name': 'James', 
                'sex': 'M', 
                'grades': { 'math': 80, 'physics': 78 } 
            },
        ]
    },
    { 
        'class': 'Year 2', 
        'student count': 28, 
        'room': 'A4',
        'info': {
            'teachers': { 
                'math': 'Alan Turing', 
                'physics': 'Vera Rubin' 
            }
        },
        'students': [
            { 'name': 'Tony', 'sex': 'M' },
            { 'name': 'Jacqueline', 'sex': 'F' },
        ]
    },
]

In [ ]:
json_list[0]

In [ ]:
json_list[0].keys()

In [ ]:
json_list[0]['class']

In [ ]:
json_list[0]['students']

Usando Pandas podemos intentar transformar un archivo json en un DataFrame estructurado:

In [ ]:
df = pd.DataFrame(json_list)
df.head()

In [ ]:
df['students']

In [ ]:
df['students'][0]

In [ ]:
df['students'][0][0]

### 3.2 Transformando un json en un DataFrame

El comando anterior no nos generó un DataFrame muy útil, pues muchos de los elementos son diccionarios adentro de una posición.

Una característica común del formato `json`, y es porque los usamos, es que pueden ser anidado: el valor de un atributo pueder ser un nuevo diccionario, o una lista de diccionarios. Este tipo de datos anidados es más conveniente de utilizar en formato "aplanado" o "flattened", de manera que pueda ser almacenado en un DataFrame. La librería `pandas` tiene la función `json_normalize()` que hace exactamente esto por nosotros. Pueden encontrar más detalles en: https://pandas.pydata.org/pandas-docs/version/1.2.0/reference/api/pandas.json_normalize.html

In [ ]:
all_students = pd.json_normalize(json_list, sep=".")
type(all_students)

In [ ]:
all_students.head()

La función `json_normalize` expande automáticamente todos los diccionarios anidados. Si tenemos también listas de diccionarios (como ocurre con `students`), hay que usar el parámetro `path_record` para expandir los diccionarios contenidos dentro de la lista.

In [ ]:
all_students = pd.json_normalize(json_list, sep="_", record_path=['students'])
all_students.head()

Esto nos permitió ordenar por alumno, pero nos quedó afuera mucha información asociada al alumno. Para agregarla debemos indicar, usando el parámetro `meta` la información adicional a copiar.

In [ ]:
all_students_grades = pd.json_normalize(json_list, sep="_", record_path=['students'], meta=['class','student count','room','info'], max_level=None)
all_students_grades.head()

Podemos expandir aún más la información, agregando los detalles por profesor.

In [ ]:
all_students_grades= pd.json_normalize(json_list, sep="_", record_path=['students'], meta=['class','student count','room',['info','teachers','math'], ['info','teachers','physics']], max_level=None)
all_students_grades.head()

In [ ]:
# almacenar la información bajo este orden en un archivo json
all_students_grades.to_json('students.json')

## 4. Lectura de un archivo json

La librería `json` también nos permite leer este tipo de formato e importar directamente a nuestro código. Hagamos el ejemplo con los datos de restaurantes de santiago de Yelp.

In [ ]:
restaurantes = json.load(open('restaurants.json','r'))
type(restaurantes)

In [ ]:
restaurantes

In [ ]:
restaurantes.keys()

In [ ]:
restaurantes['total']

In [ ]:
restaurantes['region']

In [ ]:
restaurantes['businesses'][0]

In [ ]:
df = pd.DataFrame(restaurantes['businesses'])
df.head()

## 5. Conciertos de la Filarmónica de NY

Revisemos ahora un archivo JSON más grande, con los datos de los conciertos de la Orquesta Filarmónica de Nueva York. 

El `json` de la filarmónica tiene el siguiente formato:

```
{
  "programs": [
    {
      "id": "38e072a7-8fc9-4f9a-8eac-3957905c0002", // GUID
      "programID": "3853", // NYP Local ID
      "orchestra": "New York Philharmonic",
      "season": "1842-43",
      "concerts": [
        {
           "eventType": "Subscription Season",
           "Location": "Manhattan, NY",
           "Venue": "Apollo Rooms",
           "Date": "1842-12-07T05:00:00Z",
           "Time": "8:00PM"
        },
        /* A program can have multiple concerts */
      ],
      "works": [
        {
          "ID": "8834*4", // e.g. "1234*1" - first part is the Work ID, second part is the NYP Movement ID
          "composerName": "Weber,  Carl  Maria Von",
          "workTitle": "OBERON",
          "movement": "\"Ozean, du Ungeheuer\" (Ocean, thou mighty monster), Reiza (Scene and Aria), Act II",
          "conductorName": "Timm, Henry C.",
          "soloists": [
            {
              "soloistName": "Otto, Antoinette",
              "soloistInstrument": "Soprano",
              "soloistRoles": "S"
            },
            /* more soloists, if applicable. If no soloists, this will be an empty array */
          ]
        },
        /* a program will usually have multiple works */
        {
          "ID": "0*",
          "interval": "Intermission",
          "soloists": []
        },
        /* Intermissions will also appear in the works array */
      ]
    },
    /* more programs */
  ]
}
```

In [ ]:
datos_raw = json.load(open('complete.json','r', encoding='UTF-8'))
type(datos_raw)

In [ ]:
datos_raw['programs'][0].keys()

In [ ]:
datos_raw['programs'][0]

In [ ]:
datos_raw['programs'][0]['concerts']

In [ ]:
datos_raw['programs'][0]['works']

In [ ]:
# tratemos de crear un DataFrame
nycphil = pd.json_normalize(datos_raw['programs'])
nycphil.head()

Vemos dos columnas que tienen datos anidados. Usemos la función `json_normalize` para desempacar la columna de conciertos en un nuevo DataFrame

In [ ]:
concert_data = pd.json_normalize(data=datos_raw['programs'], record_path='concerts', meta=['id', 'orchestra','programID', 'season'])
concert_data.head()

¿Cuántos conciertos han realizado desde el 7 de diciembre de 1842?

In [ ]:
len(concert_data)